In [12]:
#import
import numpy as np
import numpy.ma as ma
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from habitablePlanets import habitable_planets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


#read from CSV
exoplanets = pd.read_csv("exoplanets3.csv", sep=",")
training_data = np.array(exoplanets[:])

'''print(training_data.shape)
print(training_data)'''

#separation of values
habitable_x = np.array([np.array(training_data[0, :])])
habitable_y = np.array([0])
uninhabitable_x = np.array([np.array(training_data[0, :])])
uninhabitable_y = np.array([0])

for i in training_data:
    if i[0] in habitable_planets:
        habitable_x = np.append(habitable_x, np.array([np.array(i)]), axis = 0)
        habitable_y = np.append(habitable_y, 1)
        

for i in training_data:
    if i[0] not in habitable_planets:
        uninhabitable_x = np.append(uninhabitable_x, np.array([np.array(i)]), axis = 0)
        uninhabitable_y = np.append(uninhabitable_y, 0)

        
training_x = np.append(habitable_x[:, 1:], uninhabitable_x[:, 1:], axis = 0)
training_y = np.append(habitable_y, uninhabitable_y, axis = 0)

habitable_x = habitable_x[:, 1:]
uninhabitable_x = uninhabitable_x[:, 1:]

#synthetic data generation
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from collections import Counter

#process NaN
training_x = np.asarray(training_x).astype(np.float32)
training_x = np.where(np.isnan(training_x), ma.array(training_x, mask=np.isnan(training_x)).mean(axis=0), training_x)

oversample = SMOTE()
x_res, y_res = oversample.fit_resample(training_x, training_y)

x_train = x_res
y_train = y_res

x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size = 0.2, random_state = 0)

#nonsynthetic data
'''
x_test = np.array([np.array(training_data[0, 1:])])
y_test = np.array([0])

hab_lim = 20
cur_hab = 1

for i in training_data:
    if cur_hab >= hab_lim:
        break
    if i[0] in habitable_planets:
        y_test = np.append(y_test, 1)
        cur_hab += 1
    else:
        y_test = np.append(y_test, 0)
    x_test = np.append(x_test, np.array([np.array(i[1:])]), axis = 0)

x_test = np.asarray(x_test).astype(np.float32)
x_test = np.where(np.isnan(x_test), ma.array(x_test, mask=np.isnan(x_test)).mean(axis=0), x_test)
y_test = np.asarray(y_test).astype(np.float32)
y_test = np.where(np.isnan(y_test), ma.array(y_test, mask=np.isnan(y_test)).mean(axis=0), y_test)
'''

print(x_test.shape, y_test.shape)


(1747, 25) (1747,)


In [14]:
from sklearn.neighbors import KNeighborsClassifier
import imblearn


knn = KNeighborsClassifier(n_neighbors=10)
model = knn.fit(x_train, y_train)
y_pred = model.predict(x_test)
print("training set score: {:.4f}".format(knn.score(x_train, y_train)))
print("Validation set score: {:.4f}".format(knn.score(x_test, y_test)))



from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

training set score: 0.9828
Validation set score: 0.9794
[[827  36]
 [  0 884]]
              precision    recall  f1-score   support

           0       1.00      0.96      0.98       863
           1       0.96      1.00      0.98       884

    accuracy                           0.98      1747
   macro avg       0.98      0.98      0.98      1747
weighted avg       0.98      0.98      0.98      1747

0.9793932455638237
